In [ ]:
# default_exp models.cnn.learner

In [ ]:
# hide
import sys

sys.path.append("..")
from fastseq.all import *

In [ ]:
# export
from fastcore.all import *
from fastai.basics import *
from fastai.vision import models
from fastai.vision.learner import *
from torch.nn import Conv2d, Sequential, Module
from typing import Callable

# CNN Learner

> A CNN learner for time-series.

inspired by <https://github.com/mogwai/fastai_audio/blob/master/audio/learner.py>

TODO make transformer that makes images out of time series

In [ ]:
# export
def adapt_conv(conv: Conv2d, n_channels:int, pretrained:bool=False,
               init=None, padding_mode:str='zeros'):
    '''Create a new layer that adapts `conv` to accept `n_channels` inputs.
       Copies existing weights if `pretrained` or initialises them with `init`.'''
    if conv.in_channels == n_channels: return conv # No need to adapt    
    args = {n: getattr(conv, n) for n in ['kernel_size','stride','padding','dilation','groups']}
    bias = conv.bias is not None
    if 'padding_mode' in Conv2d.__constants__: # Padding mode added in PyTorch 1.1
        args['padding_mode'] = ifnone(padding_mode, conv.padding_mode)
    new_conv = Conv2d(n_channels, conv.out_channels, bias=bias, **args)
    if pretrained:
        exp_shape = (conv.out_channels, conv.in_channels, *conv.kernel_size)
        assert conv.weight.shape == exp_shape, f"Unexpected weights shape, expected {exp_shape}, got {conv.weight.shape}."
        new_conv.weight.data[...] = conv.weight.data[:,0:1,:,:]
        if bias: new_conv.bias.data = conv.bias.data
    elif init: init_default(new_conv, init)
    new_conv.to(conv.weight.device)
    return new_conv

In [ ]:
def adapt_model(model:Union[Module,Sequential], n_channels:int, name:str='conv1',
                   pretrained:bool=False, init=None, padding_mode:str='zeros'):
    '''Adapt a convolutional model to `n_channels` inputs and copy weights if `pretrained` or initialise with `init`.'''
    # Find direct parent of first conv layer. Could be either a Sequential or a custom Module (but not the Conv itself)
    while (isinstance(model, Sequential) and 
           isinstance(model[0], (Sequential,Module)) and
           not isinstance(model[0], Conv2d)):
        model = model[0]
    if isinstance(model, Sequential) and isinstance(model[0], Conv2d):
        conv1 = model[0]
        def update(conv): model[0] = conv
    elif isinstance(model, Module) and hasattr(model, name):
        conv1 = getattr(model, name)
        update = partial(setattr, model, name)
    else: raise TypeError(f"Could not locate first convolution layer. If it is a named layer then pass it's name, otherwise use adapt_conv.")
    update(adapt_conv(conv1, n_channels, pretrained=pretrained, init=init, padding_mode=padding_mode))


In [ ]:
mdl = create_cnn_model(models.resnet18,8,None,True)
adapt_model(mdl,9)
test_eq(mdl(torch.randn(64,9,255,255)).shape,(64,8))

In [ ]:
def ts_learner(data:TSDataLoaders, base_arch:Callable=models.resnet18, metrics=accuracy, 
                  cut:Union[int,Callable]=None, pretrained:bool=False, lin_ftrs=None, 
                  ps=0.5, custom_head:Optional[nn.Module]=None, split_on=None, 
                  bn_final:bool=False, init=nn.init.kaiming_normal_, concat_pool:bool=True, 
                  padding_mode:str='zeros', **kwargs) -> Learner:
    '''Create a learner to apply a CNN model to Time series.'''
    learn = cnn_learner(data, base_arch, cut=cut, metrics=metrics, pretrained=pretrained, loss_func=MSELossFlat(), config = {'ps':ps,
                        'custom_head':custom_head, 'bn_final':bn_final, 'init':init,
                        'concat_pool':concat_pool, **kwargs})
#     channels = _calc_channels(data)
    adapt_model(learn.model, 1, pretrained=pretrained, init=init, padding_mode=padding_mode)
#     learn.unfreeze() # Model shouldn't be frozen, unlike vision
    return learn

# Intergration Example

In [ ]:
path = untar_data(URLs.m4_daily)
data = TSDataLoaders.from_folder(path, nrows=100)
learn = ts_learner(data)


In [ ]:
learn.fit(1)

(#3) [0,0.0,'00:00']


RuntimeError: Expected 4-dimensional input for 4-dimensional weight 64 1 7 7, but got 3-dimensional input of size [16, 1, 3] instead

In [ ]:
# hide
from nbdev.export import *

notebook2script()

Converted 00_core.ipynb.
Converted 01_data.external.ipynb.
Converted 03_data.load.ipynb.
Converted 04_data.transforms.ipynb.
Converted 05_models.wavenet.ipynb.
Converted 06_models.dnn.ipynb.
Converted 08_metrics.ipynb.
Converted 09_learner.ipynb.
Converted 20_models.cnn.learner.ipynb.
Converted 21_models.cnn.transforms.ipynb.
Converted index.ipynb.
